# WL CVD graphene measurements

In [1]:
%load_ext autoreload
%autoreload 1

#### Collect all .ibw files

In [2]:
import os, shutil

owncloud_data = '/Users/nik/owncloud/'
experiment_data = 'dil_fridge/Ali/CVD, Nov. 2015_II'

local_data = '/Users/nik/Data/CVD_graphene_nov2015'

# source_data = os.path.join(owncloud_data, experiment_data)
source_data = os.path.join(owncloud_data, experiment_data)
os.chdir(local_data)

def find_ibw(directory):
    """ find *.ibw files in directory and subdirectories """
    
    ibw = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.ibw'):
                ibw.append(os.path.join(root, file))
    return ibw

def update_data_files(source_data, local_data):
    """ look in source data folder for any new *.ibw files and copy them to local_data """
    
    src = find_ibw(source_data)
    local = find_ibw(local_data)
    local = [l.split('/')[-1] for l in local]
    
    moved = []
    for s in src:
        datname = s.split('/')[-1]
        if datname in local:
            continue
        else:
            shutil.copy(os.path.join(source_data, s), os.path.join(local_data, datname))
            moved.append(s)
    return moved

print update_data_files(source_data, local_data)

[]


#### Package imports

In [3]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('/Users/nik/Dropbox/Notebooks/folklabrc.py')
import itertools
import numpy as np
import pandas as pd
%aimport igor
from igor.binarywave import Waves
import lmfit
from lmfit import Model
import pickle

:0: FutureWarning: IPython widgets are experimental and may change in the future.


#### Attempt at SRS Calibration (may be useful later)

In [4]:
def guess_range(volts):
    ranges = np.array([0.0, 0.5, 1.0, 2.0, 5.0, 10.0, 20.0, 50.0])
    ranges = zip(ranges[0:-1], ranges[1:])
    for low, high in ranges:
        if (volts*1000.0>=low) & (volts*1000.0<=high):
            return high

srs6_calibration = {0.5:0.992256, 1.0:0.992920, 2.0:0.993104, 5.0:0.993599, 10.0:0.993866, 20.0:0.993396, 50.0:1.0}
srs5_calibration = {0.5:0.993360, 1.0:0.992577, 2.0:0.992749, 5.0:0.993284, 10.0:0.993404, 20.0:0.993106, 50.0:1.0}
srs8_calibration = {0.5:0.999229, 1.0:0.998049, 2.0:0.999247, 5.0:1.000250, 10.0:1.000775, 20:1.000286, 50.0:1.0}

## Datasets

Load datasets into dataframes and save metadata as csv

In [51]:
# below are a few useful functions for loading data sets

def create_metadata_frame(datnum, sort_by = None, **kwargs):
    columns = ['datnum', 'sweeps', 'sweep_type', 'Tmix', 'V_gate', 'B_parallel', 'I_bias']
    df = pd.DataFrame(np.zeros((len(datnum),len(columns))), columns = columns)
    df['datnum'] = datnum
    for k in kwargs:
        df[k] = kwargs[k]
    df.sort(columns='datnum', axis=0, ascending=True, inplace=True)
    df = df.reset_index(drop=True)
    for ind in df.index:
        if ind not in df.index:
            continue
        if df.loc[ind, 'sweep_type']=='multiple':
            n = df.loc[ind,'sweeps']
            dat = df.loc[ind, 'datnum']
            drop = [d for d in range(dat+1, dat+sweeps)]
            df = df[~df['datnum'].isin(drop)]
    if sort_by:
        df.sort(columns=sort_by, axis=0, ascending=True, inplace=True)
        df = df.reset_index(drop=True)
    return df

def get_Tmix(df):
    for index, row in df.iterrows():
        try:
            w = Waves('dat{0}mixchtemp.ibw'.format(row['datnum']))
            row['Tmix'] = w.y.mean()
        except:
            continue
            
# I think you want to add a Tset column
# round to the nearest 10mK below 1000mK
# round to the nearest 200mK above 1000mK
            
def add_wlfit_columns(df):
    numcols = df['sweeps'].max()
    parameters = ['bo', 'ro', 'Bphi', 'Bi', 'Bstar', 'lin', 'A']         
    for p in parameters: 
        df[p] = 0.0
        df[p+'_stderr'] = 0.0
    for i in range(numcols):
        parametersn = [p+str(i) for p in parameters]
        for p in parametersn: 
            df[p] = 0.0

In [49]:
# here are a bunch of code blocks, each representing a new data set

# ### T = 6K, V_gate = 0V, multiple B_parallel 
# # Silvia: I do not think we did change the bpar here, at least I did not analyze this data.

# ### 1776mK, 100nA bias, sweep B parallel ###

# datnums = [120, 126, 132, 138, 144, 163, 169, 175, 181, 187, 
#            193, 199, 211, 227, 233, 239, 245, 251, 257, 263, 
#            269, 275, 281, 287, 293, 299]
# fields = [-0.5, -0.25, 0.0, 0.25, 0.5, 1.5, 1, 0.5, 0.3, 0.2, 0.1, 
#           0.0, 0.0, 3.0, 2.5, 2.0, 0.1, 2.75, 2.25, 1.75, 1.25, 
#           0.75, 0.1, 0.0, -0.1, -0.2]
# d = {'B_parallel':fields, 'I_bias':100.0, 'V_gate':0.0, 'Tmix':100.0, 
#       'sweep_type':'2d', 'sweeps':6}
# df = create_metadata_frame(datnums, **d)

# ### 100mK, 4nA bias, sweep B parallel ###

# datnums = [n for n in range(1439,1465+1)] # all 2d sweeps
# fields = [1.0, 0.75, 0.6, 0.5, 0.4, 0.3, 0.2, 0.18, 0.15, 0.12, 0.09, 0.06, 
#           0.05, 0.04, 0.03, 0.02, 0.015, 0.01, 0.0075, 0.005, 0.0025, 0.0, 
#           -0.005, -0.01, -0.02, -0.03, -0.04] # T
# d = {'B_parallel':fields, 'I_bias':4.0, 'V_gate':40.0, 'Tmix':100.0, 
#       'sweep_type':'2d', 'sweeps':6}
# df = create_metadata_frame(datnums, **d)

# ### T = ~16mK, V_gate = 0V, multiple B_parallel ###

# # Silvia: there were other measurements before this
# #         but I did not carefully analyze them because they had a lot of bias.

# # This was measured from Nov26 to Nov28. with 2nA/2f.5nA bias.

# bpars=[0.5,0.4,0.3,0.2,0.18,0.15,0.12,0.09,0.06
#        ,0.05,0.04,0.03,0.02,0.015,0.01,0.005,0,-0.005,-0.01,-0.02];
# filenum=[762:7:902];
# bpars=-[0.03,0.04,0.05];
# filenum=[n for n in range(902,917+1,5)];

# bpars=[0.005,0.005]
# filenum=[n for n in 918:921]

# # and one more time:

# bpars=[0.5,0.4,0.3,0.2,0.18,0.15,0.12,0.09,0.06,0.05,
#        0.04,0.03,0.02,0.015,0.01,0.0075,0.005,0.0025,0,-0.005,-0.01,-0.02];
# filenum=[n for n in range(943,1053+1, 5)]

### V_gate = 40V ###

# T = ~16mK, V_gate = 40V, multiple B_parallel

# datnums = [1223, 1222, 1221, 1224, 1219, 1229, 1218, 1228, 1217, 1196, 
#            1226, 1215, 1225, 1214, 1213, 1212, 1211, 1210, 1209, 1208,
#            1207, 1206, 1205, 1204, 1203, 1202, 1201, 1200, 1199]
# bpars=[1.0,0.5,0.4,0.3,0.2,0.18,0.15,0.12,0.09,0.06,0.05,0.04,
#        0.03,0.02,0.015,0.01,0.0075,0.005,0.0025,0,-0.005,-0.01,-0.02,-0.03,-0.04,0.009:-0.003:-0.003]
# mdata = {'sweeps':4, 'sweep_type':'2d', 'Tmix':16.0, 'V_gate':40.0, 'B_parallel':bpars, 'I_bias':1.0}
# df = create_metadata_frame(datnums, **mdata)
# get_Tmix(df)
# add_wlfit_columns(df)
# df.sort(columns='B_parallel', axis=0, ascending=True, inplace=True)
# f = 'additional_data/bpar_16mK_40V_1nA.csv'
# if os.path.isfile(f):
#     pass
# else:
#     df.to_csv(f)

### V_gate= -40V ###

# 18mK, V_gate = -40V, I_bias = 8nA

datnums = [n for n in range(1669, 1684+1)]
bpars = [0.12, 0.09, 0.06, 0.05, 0.04, 0.03, 0.02, 0.01, 0.005, 0.0, -0.005, -0.01, -0.02, -0.03, -0.04, -0.05]
mdata = {'sweeps':6, 'sweep_type':'2d', 'Tmix':18.0, 'V_gate':-40.0, 'B_parallel':bpars, 'I_bias':1.6}
df = create_metadata_frame(datnums, sort_by='B_parallel', **mdata)
get_Tmix(df)
add_wlfit_columns(df)
f = 'analysis/bpar_18mK_n40V_1p6nA.csv'
df.to_csv(f)

# 100mK, V_gate = -40V, I_bias = 8nA

datnums = [n for n in range(1582, 1612+1)]
bpars = [0.005, 0.005, 2.0, 1.5, 1.0, 0.75, 0.6, 0.5, 0.4, 0.3, 0.2, 0.18, 0.15, 0.12, 0.09, 0.06, 0.05,
         0.04, 0.03, 0.02, 0.015, 0.01, 0.0075, 0.005, 0.0025, 0.0, -0.005, -0.01, -0.02, -0.03, -0.04]
mdata = {'sweeps':2, 'sweep_type':'2d', 'Tmix':100.0, 'V_gate':-40.0, 'B_parallel':bpars, 'I_bias':8.0}
df = create_metadata_frame(datnums, sort_by='B_parallel', **mdata)
get_Tmix(df)
add_wlfit_columns(df)
f = 'analysis/bpar_100mK_n40V_8nA.csv'
df.to_csv(f)

# 200mK, V_gate = -40V, I_bias = 10nA

datnums=[n for n in range(1613,1641+1)]
bpars=[2.0, 1.5, 1.0, 0.75, 0.6, 0.5, 0.4, 0.3, 0.2, 0.18, 0.15, 0.12, 0.09, 0.06, 0.05, 0.04,
       0.03, 0.02, 0.015, 0.01, 0.0075, 0.005, 0.0025, 0.0, -0.005, -0.01, -0.02, -0.03, -0.04]
mdata = {'sweeps':2, 'sweep_type':'2d', 'Tmix':200.0, 'V_gate':-40.0, 'B_parallel':bpars, 'I_bias':10.0}
df = create_metadata_frame(datnums, **mdata)
get_Tmix(df)
add_wlfit_columns(df)
f = 'analysis/bpar_200mK_n40V_10nA.csv'
df.to_csv(f)

# 400mK, V_gate = -40V, I_bias = 15nA

datnums = [n for n in range(1642,1659+1)]
bpars =  [0.5, 0.4, 0.3, 0.2, 0.18, 0.15, 0.12, 0.09, 0.06, 0.045, 
          0.03, 0.015, 0.005, -0.005, -0.02, -0.035, -0.05, -0.08]
mdata = {'sweeps':2, 'sweep_type':'2d', 'Tmix':400.0, 'V_gate':-40.0, 'B_parallel':bpars, 'I_bias':15.0}
df = create_metadata_frame(datnums, sort_by='B_parallel', **mdata)
get_Tmix(df)
add_wlfit_columns(df)
f = 'analysis/bpar_400mK_n40V_15nA.csv'
df.to_csv(f)

# B_parallel = 5mT, V_gate = -40V,

datnums = [n for n in range(1660,1668+1)]
temps = [302, 220, 180, 150, 120, 90, 70, 50, 30]
bias = [12, 10, 10, 8, 8, 5, 4, 2, 2]
sweeps = [2, 2, 2, 2, 2, 4, 4, 6, 6]
mdata = {'sweeps':sweeps, 'sweep_type':'2d', 'Tmix':temps, 'V_gate':-40.0, 'B_parallel':0.005, 'I_bias':bias}
df = create_metadata_frame(datnums, sort_by='Tmix', **mdata)
get_Tmix(df)
add_wlfit_columns(df)
f = 'analysis/temp_varyI_5mT_n40V.csv'
df.to_csv(f)
    
# B_parallel = 5mT, sweep gate, Tmix = 100mK, I_bias = 8nA

datnums = [n for n in range(1685, 1705+1)]
gate = [g for g in range(-50, 55, 5)]
datnums.extend([n for n in range(1721, 1724+1)])
gate.extend([70, 65, 60, 55])
datnums.extend([n for n in range(1725, 1728+1)])
gate.extend([-55, -60, -65, -70])

mdata = {'sweeps':2, 'sweep_type':'2d', 'Tmix':100.0, 'V_gate':gate, 'B_parallel':0.005, 'I_bias':8.0}
df = create_metadata_frame(datnums, sort_by='V_gate', **mdata)
get_Tmix(df)
add_wlfit_columns(df)
f = 'analysis/gate_5mT_100mK_8nA.csv'
df.to_csv(f)

# B_parallel = 500mT, sweep gate, Tmix = 100mK, I_bias = 8nA

datnums = [n for n in range(1729, 1757+1)]
gate = [g for g in range(-70, 75, 5)]
mdata = {'sweeps':2, 'sweep_type':'2d', 'Tmix':100.0, 'V_gate':gate, 'B_parallel':0.005, 'I_bias':8.0}
df = create_metadata_frame(datnums, sort_by='V_gate', **mdata)
get_Tmix(df)
add_wlfit_columns(df)
f = 'analysis/gate_500mT_100mK_8nA.csv'
df.to_csv(f)

# Base temperature data as a function of V_gate, I_bias = 1.6nA, B_parallel = 5mT:

datnums = [n for n in range(1758, 1772+1)]
gate = np.linspace(70, -70, 15)
mdata = {'sweeps':6, 'sweep_type':'2d', 'Tmix':18.0, 'V_gate':gate, 'B_parallel':0.005, 'I_bias':1.6}
df = create_metadata_frame(datnums, sort_by='V_gate', **mdata)
get_Tmix(df)
add_wlfit_columns(df)
f = 'analysis/gate_5mT_18mK_1p6nA.csv'
df.to_csv(f)

# Base temperature data as a function of V_gate, I_bias = 1.6nA, B_parallel = 300mT:

datnums = [n for n in range(1773, 1777+1)]
gate = np.linspace(-70, 70, 5)
mdata = {'sweeps':6, 'sweep_type':'2d', 'Tmix':18.0, 'V_gate':gate, 'B_parallel':0.3, 'I_bias':1.6}
df = create_metadata_frame(datnums, sort_by='V_gate', **mdata)
get_Tmix(df)
add_wlfit_columns(df)
f = 'analysis/gate_300mT_18mK_1p6nA.csv'
df.to_csv(f)

# gaussian spaced sweeps +/-50mT around WL peaks at 100mK, I_bias = 1.6nA (crap)

datnums = [n for n in range(1778, 1787+1)] # one up, one down at each gate
gate = [40, 15, -10, -35, -60]
sweeps = 2
gate = np.repeat(gate, sweeps)
mdata = {'sweeps':sweeps, 'sweep_type':'multiple', 
         'Tmix':100.0, 'V_gate':gate, 'B_parallel':0.005, 'I_bias':1.6}
df = create_metadata_frame(datnums, sort_by='V_gate', **mdata)
get_Tmix(df)
add_wlfit_columns(df)
f = 'analysis/gate_5mT_100mK_1p6nA.csv'
df.to_csv(f)

datnums = [n for n in range(1788, 1797+1)]
gate = [-60, -35, -10, 15, 40]
sweeps = 2
gate = np.repeat(gate, sweeps)
mdata = {'sweeps':sweeps, 'sweep_type':'multiple', 'Tmix':100.0, 'V_gate':gate, 'B_parallel':0.6, 'I_bias':1.6}
df = create_metadata_frame(datnums, sort_by='V_gate', **mdata)
get_Tmix(df)
add_wlfit_columns(df)
f = 'analysis/gate_600mT_100mK_1p6nA.csv'
df.to_csv(f)

### add these to some other dataset above ###
# # sweep over WL peak at 1T, 100mK, 8nA (continuing the plot on page 93):
# dat1798-dat1804, gate = [-75, 50, 25, 0, 25, 50, 75]

# # (continuing the plot on page 88)
# B_parallel = 1500mT, V_gate=-40V, sweep over peak, vary T:
# files = dat1805-dat1812, dat1821
# temps = [330, 250, 200, 150, 120, 90, 70, 50, 27]
# bias = [12, 10, 10, 8, 8, 5, 4, 2, 1.6]
# sweeps = [2, 2, 2, 2, 2, 4, 4, 6, 6]
 
# At base temperature, sweep over WL peak as a function of B_parallel, I_bias = 1.6nA 
#(continuing the plot on page 86):
# fields = {1.5, 1.2, 1.0, 0.8, 0.6, 0.4, 0.3, 0.2}
# dat1813-dat1820

###            ###

# 1.5K data 

datnums = [n for n in range(1827, 1840+1)]
gate = [-60, -40, -20, 0, 20, 20, 40]
sweeps = 2
gate = np.repeat(gate, sweeps)
mdata = {'sweeps':sweeps, 'sweep_type':'multiple', 'Tmix':1500.0, 'V_gate':gate, 'B_parallel':0.005, 'I_bias':50.0}
df = create_metadata_frame(datnums, sort_by='V_gate', **mdata)
get_Tmix(df)
add_wlfit_columns(df)
f = 'analysis/gate_5mT_1500mK_50nA.csv'
df.to_csv(f)

datnums = [n for n in range(1841, 1852+1)]
gate = [40, 20, 0, -20, -40, -60]
sweeps = 2
gate = np.repeat(gate, sweeps)
mdata = {'sweeps':sweeps, 'sweep_type':'multiple', 'Tmix':1500.0, 'V_gate':gate, 'B_parallel':1.2, 'I_bias':50.0}
df = create_metadata_frame(datnums, sort_by='V_gate', **mdata)
get_Tmix(df)
add_wlfit_columns(df)
f = 'analysis/gate_1200mT_1500mK_50nA.csv'
df.to_csv(f)

### Fitting routines

In [52]:
### fitting to WL peaks ###

e=1.60217646e-19;  mstar=8.724125937209915e-32; h=6.626068e-34; G_0=7.45e-5; hbar=h/2.0/np.pi;
# D=5.106250385935459e-03 # diffusion constant

from scipy.special import psi

# basic fit function
def wlcorr(b, bo, ro, Bphi, Bi, Bstar, lin, A):
    """ Weak localization peak fitting function. Adapted from Igor code. """
    b=b-bo

    z1=np.array(abs(b)/Bphi, dtype=np.float64)
    F1=np.log(z1)+psi(0.5+1/z1)

    z2=np.array(abs(b)/(Bphi+2*Bi), dtype=np.float64)
    F2=np.log(z2)+psi(0.5+1/z2)

    z3=np.array(abs(b)/(Bphi+Bstar), dtype=np.float64)
    F3=np.log(z3)+psi(0.5+1/z3)

    return (lin*b) + ro - A*ro**2.0*(e*e/np.pi/h*(F1-F2-2*F3))

### parabola to fit curvature only ###
def parabola(b, bo, ro, a):
    """ Inverse parabola for fitting  """
    return ro - a*np.power(b - bo, 2)

In [54]:
def find_nearest(array, value):
    idx = (np.abs(array-value)).argmin()
    return idx

def round_to(n, precision):
    """ http://stackoverflow.com/questions/4265546/python-round-to-nearest-05 """
    correction = 0.5 if n >= 0 else -0.5
    return int( n/precision+correction ) * precision

def import_2d_dataset(datnum, bias, sample, measured_current = True):
    """ lock in configuration for all of my 2d data is:
    
        g6: Rxx top
        g5: Rxx top
        g8: Rxx bottom
        g9: Current top 
        
        Returns: rxx dataframe with x and y components """
    
    if sample=='top':
        rxx = (Waves('dat{0:d}g6x2d.ibw'.format(int(datnum))).as_dataframe(sharex=True)
               +Waves('dat{0:d}g5x2d.ibw'.format(int(datnum))).as_dataframe(sharex=True))/2.0
        if measured_current == True:
            bias = Waves('dat{0:d}g9x2d.ibw'.format(int(datnum))).y.mean().mean()*1000.0
        scale = (1e-3)*(1e9)/(bias)/(1.5)
        rxx *= scale
    elif sample=='bottom':
        rxx = Waves('dat{0:d}g8x2d.ibw'.format(int(datnum))).as_dataframe(sharex=True)
        scale = (1e-3)*(1e9)/(bias)/(1.5)
        rxx *= scale
    rxx.iloc[:, 2::2] = rxx.iloc[:, 2::2].values[::-1, :] # flip downsweeps to correct direction
    rxx.x = Waves('dat{0:d}fields.ibw'.format(int(datnum))).y
    return rxx

# def import_multiple_dataset(datnum, bias, sample, measured_current = True):
#     """ lock in configuration for all of my 2d data is:
    
#         ???? 

#         Returns: rxx dataframe with x and y components """
    

def wl_fit(b, rxx, width, height, fix = None, ro=None, bcenter = None):
    """ fit a single b_perpendicular (b) and r_xx set to the WL model.
    
        Inputs:  width -- range of data over which to fit
                 fix -- dictionary of parameters to fix and their values
                 ro -- initial guess at ro
                 bcenter -- initial guess at bcenter 
        Outputs: results -- lmfit results object """

    # create model
    model = Model(wlcorr, independent_vars=['b'])
    params = model.make_params()
    method = 'nelder'

    if not bcenter:
        bcenter = b.mean()
    if not ro:
        ro = rxx[find_nearest(b, bcenter)]
    
    ### data selection criteria ###
    right = bcenter+width
    left = bcenter-width 
    bottom = (1-height)*ro

    bi = (b > left) & (b < right)
    ri = (rxx > bottom)
    inds = bi & ri

    b = b[inds]
    rxx = rxx[inds]
    ### end data selection ###

    # always vary
    params['bo'].value = bcenter
    params['ro'].value = round_to(ro, 0.5)
    params['Bphi'].value = 0.15

    # sometimes vary
    params['Bi'].value = 1.8
    params['Bstar'].value = 2.5e7
    params['lin'].value = 0.0027
    params['A'].value = 1.00 
        
    if fix:
        for p in fix:
            params[p].vary = False
            params[p].value = fix[p]

    result = model.fit(rxx, params, b=b, method = method)
    return result

def get_single_wlfits(df, width, height, fix = None, sample='top'):
    """ fill dataframe with fit results for single sweeps """

    for ind in df.index:
        datnum = df.loc[ind,'datnum'] # get dat file number
        sweeps = df.loc[ind,'sweeps']

        # import data based on sweep type
        # the y data is not scaled at all
        if df.loc[ind,'sweep_type']=='2d':
            rxx = import_2d_dataset(datnum, bias=df.loc[ind,'I_bias'], sample=sample)
        
        for n in range(sweeps):
            c = 'y{0:d}'.format(n)
            result = wl_fit(rxx.x, rxx[c], width, height, fix = fix, ro=None, bcenter = None)
            for p in result.params:
                df.loc[ind,p+str(n)] = result.values[p]
                
def get_average_wlfits(df, width, height, fix=None, sample='top'):
    
    for ind in df.index:
        datnum = df.loc[ind,'datnum'] # get dat file number
        sweeps = df.loc[ind,'sweeps']
        
        # import data based on sweep type
        # the y data is not scaled at all
        if df.loc[ind,'sweep_type']=='2d':
            rxx = import_2d_dataset(datnum, bias=df.loc[ind,'I_bias'], sample=sample)
                                    
        y_cols = ['y{0:d}'.format(n) for n in range(sweeps)]
        ro_cols = ['ro{0:d}'.format(n) for n in range(sweeps)]
        
        ydata = rxx[y_cols].values
        ydata = ydata.reshape(-1,1)
               
        xdata = np.tile(rxx.x.values, (sweeps, 1)).transpose()
        xdata = xdata - df.loc[ind,ro_cols].values # subtract centers values
        xdata = xdata + df.loc[ind, ro_cols].mean() # a
        xdata = xdata.reshape(-1,1)
        sort_ind = np.argsort(xdata, axis=None)
        
        xdata = xdata[sort_ind]
        ydata = ydata[sort_ind]
        
        xdata = pd.rolling_mean(xdata, sweeps, center=True)
        ydata = pd.rolling_mean(ydata, sweeps, center=True)
        ydata = ydata[~np.isnan(ydata)]
        xdata = xdata[~np.isnan(xdata)]
        
        result = wl_fit(xdata, ydata, width, height, fix = fix, ro=None, bcenter = None)
        for p in result.params:
            pcols = ['{0}{1:d}'.format(p,n) for n in range(sweeps)]
            df.loc[ind,p] = result.values[p]
            df.loc[ind,p+'_stderr'] = df.loc[ind,pcols].values.std()
            
### newer stuff, works better ###
# here is a great example someone wrote on stackoverflow: 
# http://stackoverflow.com/questions/20339234/python-and-lmfit-how-to-fit-multiple-datasets-with-shared-parameters

# return fit for dataset i
def wlcorr_dataset(params, i, b):
    """ Weak localization peak fitting function. Adapted from Igor code. """
    bo = params['bo{0:d}'.format(i)]
    ro = params['ro{0:d}'.format(i)]
    Bphi = params['Bphi{0:d}'.format(i)]
    Bi = params['Bi{0:d}'.format(i)]
    Bstar = params['Bstar{0:d}'.format(i)]
    lin = params['lin{0:d}'.format(i)]
    A = params['A{0:d}'.format(i)]
    return wlcorr(b, bo, ro, Bphi, Bi, Bstar, lin, A)

def objective(params, b, data):
    """ calculate total residual for fits to several data sets held
    in a 2-D array, and modeled by Gaussian functions"""
    nb, ndata = data.shape
    resid = np.zeros(data.shape)
    # make residual per data set
    for i in range(ndata):
        resid[:, i] = data[:, i] - wlcorr_dataset(params, i, b)
    # now flatten this to a 1D array, as minimize() needs
    return resid.flatten()

def wl_fit_simultaneous(x, y, fix = None):

    # create parameters, one per data set
    fit_params = Parameters()
    bcenter = x.mean()
    rpeak = y[find_nearest(b, bcenter)].mean()
    guess = {'bo':bcenter, 'ro':rpeak, 'Bphi':0.15, 'Bi': 1.75, 'Bstar': 2500000.0, 'lin': 0.0027, 'A': 1.08}
    for i in range(y.shape[1]):
        fit_params.add('bo{0:d}'.format(i), value = guess['bo'])
        fit_params.add('ro{0:d}'.format(i), value = guess['ro'])
        fit_params.add('Bphi{0:d}'.format(i), value = guess['Bphi'])
        fit_params.add('Bi{0:d}'.format(i), value = guess['Bi'])
        fit_params.add('Bstar{0:d}'.format(i), value = guess['Bstar'])
        fit_params.add('lin{0:d}'.format(i), value = guess['lin'])
        fit_params.add('A{0:d}'.format(i), value = guess['A'])

    # fix any parameters
    if fix:
        for p in fix:
            for i in range(y.shape[1]):
                fit_params[p+str(i)].vary = False
                fit_params[p+str(i)].value = fix[p]

    # add constraints for all variables but bo and ro
    for i in range(y.shape[1])[1:]:
        fit_params['Bphi{0:d}'.format(i)].expr = 'Bphi0'
        fit_params['Bi{0:d}'.format(i)].expr = 'Bi0'
        fit_params['Bstar{0:d}'.format(i)].expr = 'Bstar0'
        fit_params['lin{0:d}'.format(i)].expr = 'lin0'
        fit_params['A{0:d}'.format(i)].expr = 'A0'

    # # run the global fit to all the data sets
    m = minimize(objective, fit_params, args=(x, y))
    return fit_params

def simultaneous_wlfits(df, fix = None, sample='top'):
    """ fill dataframe with fit results for single sweeps """

    for ind in df.index:
        datnum = df.loc[ind,'datnum'] # get dat file number
        sweeps = df.loc[ind,'sweeps']

        # import data based on sweep type
        # the y data is not scaled at all
        if df.loc[ind,'sweep_type']=='2d':
            rxx = import_2d_dataset(datnum, bias=df.loc[ind,'I_bias'], sample=sample)
        
            params = wl_fit_simultaneous(rxx.x.values, rxx.iloc[:,1:].values, fix = fix)
            for p in params:
                df.loc[ind,p] = params[p].value

In [55]:
def import_multiple_dataset(datnum, sweeps, bias, sample, measured_current = True):
    """ lock in configuration for all of my 2d data is:
    
        ???? 

        Returns: rxx dataframe with x and y components """
    dats = [d for d in range(datnum, datnum+sweeps)]
    
    
    if sample=='top':
        rxx = (Waves('dat{0:d}g6x.ibw'.format(int(datnum))).as_dataframe(sharex=True)
               +Waves('dat{0:d}g5x.ibw'.format(int(datnum))).as_dataframe(sharex=True))/2.0
        if measured_current == True:
            bias = Waves('dat{0:d}g9x.ibw'.format(int(datnum))).y.mean().mean()*1000.0
        scale = (1e-3)*(1e9)/(bias)/(1.5)
        rxx *= scale
    elif sample=='bottom':
        rxx = Waves('dat{0:d}g8x.ibw'.format(int(datnum))).as_dataframe(sharex=True)
        scale = (1e-3)*(1e9)/(bias)/(1.5)
        rxx *= scale
        
        
    rxx.iloc[:, 2::2] = rxx.iloc[:, 2::2].values[::-1, :] # flip downsweeps to correct direction
    rxx.x = Waves('dat{0:d}fields.ibw'.format(int(datnum))).y